In [1]:
import sys
import ok
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import PIL.Image as img
import numpy as np
import cv2
from IPython.display import clear_output
%matplotlib tk

In [2]:
dev = ok.okCFrontPanel()
deviceCount = dev.GetDeviceCount()
for i in range(deviceCount):
    print 'Device[{0}] Model: {1}'.format(i, dev.GetDeviceListModel(i))
    print 'Device[{0}] Serial: {1}'.format(i, dev.GetDeviceListSerial(i))

Device[0] Model: 21
Device[0] Serial: 143400091C


In [3]:
dev = ok.okCFrontPanel()
dev.OpenBySerial("")
error = dev.ConfigureFPGA("ok_imager.bit")
# Its a good idea to check for errors here

In [4]:
# IsFrontPanelEnabled returns true if FrontPanel is detected.
if True == dev.IsFrontPanelEnabled():
    print "FrontPanel host interface enabled."
else:
    sys.stderr.write("FrontPanel host interface not detected.")

FrontPanel host interface enabled.


In [5]:
row = 160
N_adc = 12
N_mux = 47
col = N_adc*N_mux
col_fifo = 564
count = 1
datain128 = bytearray(131072)
datain1 = bytearray(90240)

In [6]:
#im = np.array(img.open('imTest.jpg'))
#imgplot = plt.imshow(im)
im = np.zeros(shape=(row ,col))
#plt.figure(3)
#plt.imshow(im).set_cmap('gray')
#plt.show()

In [7]:
# assert reset signal to initialize the FIFO.
dev.SetWireInValue(0x10, 0xff, 0x01);
dev.UpdateWireIns();
# deactivate reset signal and activate counter.
dev.SetWireInValue(0x10, 0x00, 0x01);
dev.UpdateWireIns();   

In [8]:
# Trigger the counter
dev.ActivateTriggerIn(0x53, 0x01)
cv2.namedWindow("imFull",cv2.WINDOW_NORMAL)
cv2.namedWindow("im",cv2.WINDOW_NORMAL)
while True:
    # Check for FIFO flag
    dev.UpdateTriggerOuts()
    # If the FIFO is full, read everything and display one frame only and exit the while loop
    if dev.IsTriggered(0x6A, 0x01) == True:
        # print 'FIFO full! ', count, ' times'
        count = count + 1
        dev.ReadFromPipeOut(0xA0, datain128)
        for i in range(row):
            im[i] = datain128[i*col:i*col+col]
        im = im/255
        cv2.imshow('imFull',im)
        dev.SetWireInValue(0x10, 0xff, 0x01);
        dev.UpdateWireIns();
        dev.SetWireInValue(0x10, 0x00, 0x01);
        dev.UpdateWireIns(); 
        cv2.waitKey(5)
        # dev.ActivateTriggerIn(0x53, 0x02)
        # break
    # If one frame is ready in FIFO
    if dev.IsTriggered(0x6A, 0x04) == True:
        dev.ReadFromPipeOut(0xA0, datain1)
        for i in range(row):
            for k in range(N_adc):
                for j in range(N_mux):
                    im[i][k*N_mux+j] = datain1[i*col+j*N_adc+k]
        im = im/255
        cv2.imshow('im',im)
        cv2.waitKey(5)

KeyboardInterrupt: 

In [9]:
print i
print j
print k
print col
print i*col+j*N_adc+k*4
print k*N_mux+j

0
0
5
564
20
235


In [10]:
print im[0]*255

[  0.00000000e+00   2.04000000e+03   5.10000000e+02   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   5.10000000e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.04000000e+03
   5.10000000e+02   2.04000000e+03   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   5.10000000e+02   0.00000000e+00   2.04000000e+03   5.10000000e+02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   5.10000000e+02
   0.00000000e+00   5.10000000e+02   0.00000000e+00   6.50250000e+04
   6.50250000e+04   6.50250000e+04   6.50250000e+04   6.50250000e+04
   6.50250000e+04   6.50250000e+04   6.50250000e+04   6.50250000e+04
   6.50250000e+04   6.50250000e+04

In [11]:
print im[10]*255

[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.   53.   56.
   59.   56.   58.   58.   58.   59.   59.   59.   59.   59.   59.   59.
   58.   57.   57.   57.   58.   57.   59.   58.   58.   57.   58.   57.
   59.   58.   58.   59.   58.   58.   59.   58.   59.   56.   58.   60.
   59.   58.   59.   58.   59.   59.   59.   58.   59.   53.   60.  154.
   73.  160.   79.  162.   79.  155.   79.  162.  182.   92.   61.  145.
  176.   86.   60.  155.   77.  156.   77.  159.   

In [12]:
print im[100]*255

[   0.    8.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    8.    0.    0.    0.    0.    8.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.
  255.  255.  255.  255.  255.  255.  255.  255.  255.  255.   54.   59.
   58.   60.   58.   58.   60.   61.   59.   60.   59.   60.  131.   70.
   60.   59.   59.   61.   62.   60.  135.   71.   62.   58.   58.   57.
   59.   60.   58.   57.   60.   60.   59.   58.   58.   57.   58.   58.
   59.   58.   60.   58.   59.   59.   58.   58.   58.   54.   61.  152.
   76.  153.   78.  163.   82.  157.   81.  164.   78.  159.   79.  150.
   75.  152.   78.  161.   81.  162.   79.  164.   

In [13]:
print im[100][1:10]*255
print col

[ 8.  0.  0.  0.  0.  0.  0.  0.  0.]
564
